In [7]:
import torch.nn as nn
from torch.nn import *
import torch
class Residual(nn.Module):

    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):

        return self.fn(x) + x

def ConvMixer(dim, depth, kernel_size=9, patch_size=7, n_classes=1000):

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
        Residual(nn.Sequential(
        nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
        nn.GELU(),
        nn.BatchNorm2d(dim))),
        nn.Conv2d(dim, dim, kernel_size=1),
        nn.GELU(),nn.BatchNorm2d(dim)) for i in range(depth)],
        nn.AdaptiveAvgPool2d((1,1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes))

def ConvMixer(h,d,k,p,n):
    '''
     h = dim
     d = depth
     p = patch_size
     k = kernel_size
     n = classes
    '''
    S,C,A=Sequential,Conv2d,lambda x:S(x,GELU(),BatchNorm2d(h))
    R=type('',(S,),{'forward':lambda s,x:s[0](x)+x})
    return S(A(C(3,h,p,p)),*[S(R(A(C(h,h,k,groups=h,padding=k//2))),A(C(h,h,1))) for i in range(d)],AdaptiveAvgPool2d(1),Flatten(),Linear(h,n))

In [10]:
model = ConvMixer(128,12,7,4,10)

In [11]:
model(torch.rand(1,3,224,224))

tensor([[ 0.0176, -0.0163, -0.0297, -0.0864, -0.0406,  0.0803, -0.0438, -0.0213,
         -0.0640, -0.0805]], grad_fn=<AddmmBackward0>)

In [12]:
inputs = torch.rand(1,3,224,224)
model.eval()

torch.onnx.export(model,inputs,'test.onnx')